# File 1:Firms' Preprocessing

Working on firm level data <br>
- This will generate patent_firm_full_clean.pkl file

In [1]:
import pandas as pd
import nltk
import numpy as np
import requests
import pickle

In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
patents_new=pd.read_csv('patent_all_en_new.csv')

- appln                     The identifier of the application (with letter at the end, seems Google omits that)
- appln_id                  The (internal) patstat identifier of the application
- family_id                 The (internal) patstat identifier of the family that the application is a member of sets  - sets                      Indicate which sets (fuel-cell = 1, electric  vehicle = 2, both = 3) was matched
- direct                    Indicate whether this application was found directly (title/abstract/cpc/ipc) or      indirectly (family association)
- granted                   Whether or not the application was granted
- earliest_appln            Per family, the earliest application
- earliest_filing_date      The date of the family's earliest application
- publn                     Identifiers of publications associated with this application (*)
- publn_date                The dates of associated publications (*)
- publn_first_grant         Indicates for each publication if its the first one with granted status
- num_cpc_class             The number of CPC classes for this application
- cpc_class                 The CPC class symbols for this application
- num_ipc_class             The number of IPC classes for this application
- ipc_class                 The IPC class symbols for this application
- num_cited_publn           The number of forward citations (this applications refers to other publications)
- cited_publn_date          The dates of cited publications (**)
- cited_publn               The identifiers of cited publications (**)
- cited_appln               The identifiers of the applications  associated with cited publications (**)
- cited_appln_id            The (internal) patstat identifiers of the  applications associated with cited publications (**)
- cited_family_id           The (internal) patstat identifiers of the  family associated with cited publications (**)
- num_cited_family          The total number of distinct families  associated with cited publications (***)
- num_citing_publn          The number of backward citations (other  publications refer to this application)
- citing_publn_date         The dates of citing publications (**)
- citing_publn              The identifiers of citing publications (**)
- citing_appln              The identifiers of the applications  associated with citing publications (**)
- citing_appln_id           The (internal) patstat identifiers of the  applications associated with citing publications (**)
- citing_family_id          The (internal) patstat identifiers of the  family associated with citing publications (**)
- num_citing_family         The total number of distinct families  associated with citing publications (***)
- earliest_applt            The name of the (first) applicant of the  earliest publication of this application 
- current_applt             The name of the (first) applicant of the most recent publication of this application
- appln_title               The title of this application

>>
>> (*/**) These are one per line, in consistent order


>> (***) These seem to be the numbers that google uses to count citations. You can get this from the export if you aggregate all applications within a family and discard duplicates.


In [4]:
patents_new.head()

,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,publn,...,citing_publn_date,citing_publn,citing_appln,citing_appln_id,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr
0,WO9105114W,47158713,1239277,2,1,1,US55592190A,1990-07-19,1993-02-18,WO9303532A1,...,2002-07-11,WO0199222A3,WO0119827W,45621927,24395918,30,LUZ ELECTRIC FUEL ISRAEL LTD.,LUZ ELECTRIC FUEL ISRAEL LTD.,ELECTRICAL ENERGY SYSTEM,An electrical energy system including an elect...
1,US55592190A,51797454,1239277,2,1,1,US55592190A,1990-07-19,1992-06-09,US5121044A,...,1993-05-04\n1994-03-09\n1994-09-20\n1995-07-18...,US5208526A\nEP0557287A4\nUS5349535A\nUS5434020...,US63645090A\nEP91915464A\nUS96399792A\nUS15296...,52393850\n16916549\n54353461\n48212422\n457802...,27071024\n1239277\n25508001\n22545199\n2636904...,30,"Luz Electric Fuel Israel, Ltd.","Luz Electric Fuel Israel, Ltd.",Electrical energy system,An electrical energy system including an elect...
2,EP96927918A,17205496,1341188,4,0,0,JP4059995A,1995-02-28,1998-08-12,EP0857862A1,...,2002-01-02\n2002-01-10\n2002-08-29\n2017-12-06,EP1167708A2\nDE10029978A1\nDE10105233A1\nEP220...,EP01115227A\nDE10029978A\nDE10105233A\nEP08842...,15814252\n12936842\n12970055\n57178895,7646156\n7646156\n7672964\n40580359,31,"SUMITOMO ELECTRIC INDUSTRIES, LTD.","SUMITOMO ELECTRIC INDUSTRIES, LTD.",PARTICULATE TRAP FOR A DIESEL ENGINE,An object is to provide a particulate trap for...
3,JP4059995A,27121457,1341188,4,1,0,JP4059995A,1995-02-28,1996-09-10,JPH08229330A,...,2010-09-27\n2011-01-27\n2013-05-02\n2014-03-18...,KR20100103462A\nJP2011502743A\nJP2013078764A\n...,KR20107010572A\nJP2010531202A\nJP2012279486A\n...,328440678\n332201837\n406999716\n378212937\n41...,40580359\n40580359\n40580359\n40580359\n405803...,31,SUMITOMO ELECTRIC IND LTD,SUMITOMO ELECTRIC IND LTD,PARTICULATE TRAP FOR DIESEL ENGINE,PURPOSE: To obtain a trap by which the leakage...
4,WO9602405W,30070806,1341188,4,0,0,JP4059995A,1995-02-28,1998-03-05,WO9809059A1,...,NaN,NaN,NaN,NaN,NaN,31,"Sumitomo Electric Industries, Ltd.","Sumitomo Electric Industries, Ltd.",PARTICULATE TRAP FOR A DIESEL ENGINE,A particulate trap for a diesel engine designe...


In [118]:
#patents_new[patents_new.current_applt=='TOYOTA MOTOR CORP']

In [119]:
#ri[ri.violation == 'Speeding'].driver_gender.value_counts()

In [120]:
assignee=patents_new['current_applt']

In [121]:
# remove punctuation
assignee = patents_new['current_applt'].str.replace('[^\w\s]','')

In [122]:
assignee = patents_new['current_applt'].replace(".", "") 

In [123]:
assignee = patents_new['current_applt'].str.replace('\[.*?\]','')

In [124]:
assignee = patents_new['current_applt'].str.replace(",","")

In [125]:
assignee=patents_new['current_applt'].str.split()

In [126]:
assignee=patents_new['current_applt'].str.lower()

In [127]:
patents_new['assignee']=assignee

In [128]:
patents_new['assignee'].value_counts()

toyota motor corp                                                   3602
toyota jidosha kabushiki kaisha                                     3543
robert bosch gmbh                                                   2653
hyundai motor company                                               2230
honda motor co ltd                                                  1903
nissan motor co ltd                                                 1374
honda motor co., ltd.                                               1196
ford global technologies, llc                                       1192
nissan motor co., ltd.                                              1140
honda giken kogyo kabushiki kaisha                                   766
denso corp                                                           603
peugeot citroen automobiles sa                                       598
toyota motor corporation                                             474
mitsubishi motors corp                             

# cleaning data

In [129]:
assignee=patents_new['current_applt'].str.replace('LUZ ELECTRIC FUEL ISRAEL LTD','luz electric fuel israel ltd')

In [130]:
assignee=patents_new['current_applt'].str.replace('Luz Electric Fuel Israel Ltd','Luz Electric Fuel Israel Ltd')

### Solving toyota duplication

In [131]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota motor co ltd','toyota motor corp')

In [132]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota jidosha kabushiki kaisha','toyota motor corp')

In [133]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota motor co., ltd.','toyota motor corp')

In [134]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota motor corporation','toyota motor corp')

In [135]:
patents_new['assignee'].value_counts()

toyota motor corp                                                   7991
robert bosch gmbh                                                   2653
hyundai motor company                                               2230
honda motor co ltd                                                  1903
nissan motor co ltd                                                 1374
honda motor co., ltd.                                               1196
ford global technologies, llc                                       1192
nissan motor co., ltd.                                              1140
honda giken kogyo kabushiki kaisha                                   766
denso corp                                                           603
peugeot citroen automobiles sa                                       598
mitsubishi motors corp                                               457
hitachi, ltd.                                                        408
denso corporation                                  

### For honda

In [136]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda motor co ltd','honda motor co ltd')

In [137]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda giken kogyo kabushiki kaisha','honda motor co ltd')

In [138]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda motor co., ltd.','honda motor co ltd')

In [139]:
patents_new['assignee'].value_counts()

toyota motor corp                                                   7991
honda motor co ltd                                                  3865
robert bosch gmbh                                                   2653
hyundai motor company                                               2230
nissan motor co ltd                                                 1374
ford global technologies, llc                                       1192
nissan motor co., ltd.                                              1140
denso corp                                                           603
peugeot citroen automobiles sa                                       598
mitsubishi motors corp                                               457
hitachi, ltd.                                                        408
denso corporation                                                    337
daimlerchrysler ag                                                   335
continental automotive gmbh                        

### nissan

In [140]:
patents_new['assignee']=patents_new['assignee'].str.replace('nissan motor co., ltd.','nissan motor co ltd')

In [141]:
patents_new['assignee'].value_counts()

toyota motor corp                                                              7991
honda motor co ltd                                                             3865
robert bosch gmbh                                                              2653
nissan motor co ltd                                                            2515
hyundai motor company                                                          2230
ford global technologies, llc                                                  1192
denso corp                                                                      603
peugeot citroen automobiles sa                                                  598
mitsubishi motors corp                                                          457
hitachi, ltd.                                                                   408
denso corporation                                                               337
daimlerchrysler ag                                                          

### mitsubishi

In [142]:
patents_new['assignee']=patents_new['assignee'].str.replace('mitsubishi jidosha kogyo kabushiki kaisha','mitsubishi motors corp')

In [143]:
patents_new['assignee']=patents_new['assignee'].str.replace('mitsubishi electric corporation','mitsubishi electric corp')

In [144]:
patents_new['assignee']=patents_new['assignee'].str.replace('mitsubishi denki kabushiki kaisha','mitsubishi motors corp')

In [145]:
patents_new['assignee'].value_counts()

toyota motor corp                                                   7991
honda motor co ltd                                                  3865
robert bosch gmbh                                                   2653
nissan motor co ltd                                                 2515
hyundai motor company                                               2230
ford global technologies, llc                                       1192
mitsubishi motors corp                                               875
denso corp                                                           603
peugeot citroen automobiles sa                                       598
mitsubishi electric corp                                             474
hitachi, ltd.                                                        408
denso corporation                                                    337
daimlerchrysler ag                                                   335
continental automotive gmbh                        

### hyundai

In [146]:
patents_new['assignee']=patents_new['assignee'].str.replace('hyundai motor company','hyundai motor co ltd')

In [147]:
patents_new['assignee']=patents_new['assignee'].str.replace('hyundai motor company','hyundai motor co ltd')

In [148]:
patents_new['assignee'].value_counts()

toyota motor corp                                                   7991
honda motor co ltd                                                  3865
robert bosch gmbh                                                   2653
nissan motor co ltd                                                 2515
hyundai motor co ltd                                                2447
ford global technologies, llc                                       1192
mitsubishi motors corp                                               875
denso corp                                                           603
peugeot citroen automobiles sa                                       598
mitsubishi electric corp                                             474
hitachi, ltd.                                                        408
denso corporation                                                    337
daimlerchrysler ag                                                   335
continental automotive gmbh                        

In [149]:
patents_new['assignee']=patents_new['assignee'].str.replace('hitachi, ltd.','hitachi ltd')

In [150]:
patents_new['assignee'].value_counts()[:50]

toyota motor corp                                     7991
honda motor co ltd                                    3865
robert bosch gmbh                                     2653
nissan motor co ltd                                   2515
hyundai motor co ltd                                  2447
ford global technologies, llc                         1192
mitsubishi motors corp                                 875
hitachi ltd                                            715
denso corp                                             603
peugeot citroen automobiles sa                         598
mitsubishi electric corp                               474
denso corporation                                      337
daimlerchrysler ag                                     335
continental automotive gmbh                            316
siemens aktiengesellschaft                             285
gm global technology operations llc                    285
yamaha hatsudoki kabushiki kaisha                      2

### ford

In [151]:
patents_new['assignee']=patents_new['assignee'].str.replace('ford global tech llc','ford global technologies inc')

In [152]:
patents_new['assignee']=patents_new['assignee'].str.replace('ford global technologies, inc.','ford global technologies inc')

In [153]:
patents_new['assignee']=patents_new['assignee'].str.replace('ford global technologies, llc','ford global technologies inc')

In [154]:
patents_new['assignee']=patents_new['assignee'].str.replace('ford global technologies, llc','ford global technologies inc')

In [155]:
patents_new['assignee']=patents_new['assignee'].str.replace('ford global technologies llc','ford global technologies inc')

In [156]:
patents_new['assignee'].value_counts()[:50]

toyota motor corp                                     7991
honda motor co ltd                                    3865
robert bosch gmbh                                     2653
nissan motor co ltd                                   2515
hyundai motor co ltd                                  2447
ford global technologies inc                          1652
mitsubishi motors corp                                 875
hitachi ltd                                            715
denso corp                                             603
peugeot citroen automobiles sa                         598
mitsubishi electric corp                               474
denso corporation                                      337
daimlerchrysler ag                                     335
continental automotive gmbh                            316
siemens aktiengesellschaft                             285
gm global technology operations llc                    285
yamaha hatsudoki kabushiki kaisha                      2

### clearing . and ,

In [157]:
patents_new['assignee'] = patents_new['assignee'].str.replace(",","")

In [158]:
patents_new['assignee'] = patents_new['assignee'].str.replace(".","")

In [159]:
patents_new['assignee'].value_counts()[:50]

toyota motor corp                                     7997
honda motor co ltd                                    3899
robert bosch gmbh                                     2659
nissan motor co ltd                                   2596
hyundai motor co ltd                                  2532
ford global technologies inc                          1671
mitsubishi motors corp                                 899
hitachi ltd                                            759
denso corp                                             640
renault sas                                            632
peugeot citroen automobiles sa                         626
mitsubishi electric corp                               513
daimlerchrysler ag                                     338
denso corporation                                      337
continental automotive gmbh                            316
aisin aw co ltd                                        306
suzuki motor corp                                      2

### renault sas vs renault

In [160]:
patents_new['assignee']=patents_new['assignee'].str.replace('renault','renault sas')

In [161]:
patents_new['assignee']=patents_new['assignee'].str.replace('renault sas sas','renault sas')

In [162]:
patents_new['assignee'].value_counts()[:100]

toyota motor corp                              7997
honda motor co ltd                             3899
robert bosch gmbh                              2659
nissan motor co ltd                            2596
hyundai motor co ltd                           2532
ford global technologies inc                   1671
mitsubishi motors corp                          899
renault sas                                     760
hitachi ltd                                     759
denso corp                                      640
peugeot citroen automobiles sa                  626
mitsubishi electric corp                        513
daimlerchrysler ag                              338
denso corporation                               337
continental automotive gmbh                     316
aisin aw co ltd                                 306
suzuki motor corp                               299
gm global technology operations llc             291
siemens aktiengesellschaft                      285
yamaha hatsu

### Removing really lower patent ones now

In [163]:
patents_new['assignee']=patents_new['assignee'].str.replace('nissan motor company limited','nissan motor co ltd')

In [164]:
patents_new['assignee']=patents_new['assignee'].str.replace('aisin seiki kabushiki kaisha','aisin aw co ltd')

In [165]:
patents_new['assignee']=patents_new['assignee'].str.replace('hyundai motor co ltd co ltd','hyundai motor co ltd')

In [166]:
patents_new.head()

,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,publn,...,citing_publn,citing_appln,citing_appln_id,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr,assignee
0,WO9105114W,47158713,1239277,2,1,1,US55592190A,1990-07-19,1993-02-18,WO9303532A1,...,WO0199222A3,WO0119827W,45621927,24395918,30,LUZ ELECTRIC FUEL ISRAEL LTD.,LUZ ELECTRIC FUEL ISRAEL LTD.,ELECTRICAL ENERGY SYSTEM,An electrical energy system including an elect...,luz electric fuel israel ltd
1,US55592190A,51797454,1239277,2,1,1,US55592190A,1990-07-19,1992-06-09,US5121044A,...,US5208526A\nEP0557287A4\nUS5349535A\nUS5434020...,US63645090A\nEP91915464A\nUS96399792A\nUS15296...,52393850\n16916549\n54353461\n48212422\n457802...,27071024\n1239277\n25508001\n22545199\n2636904...,30,"Luz Electric Fuel Israel, Ltd.","Luz Electric Fuel Israel, Ltd.",Electrical energy system,An electrical energy system including an elect...,luz electric fuel israel ltd
2,EP96927918A,17205496,1341188,4,0,0,JP4059995A,1995-02-28,1998-08-12,EP0857862A1,...,EP1167708A2\nDE10029978A1\nDE10105233A1\nEP220...,EP01115227A\nDE10029978A\nDE10105233A\nEP08842...,15814252\n12936842\n12970055\n57178895,7646156\n7646156\n7672964\n40580359,31,"SUMITOMO ELECTRIC INDUSTRIES, LTD.","SUMITOMO ELECTRIC INDUSTRIES, LTD.",PARTICULATE TRAP FOR A DIESEL ENGINE,An object is to provide a particulate trap for...,sumitomo electric industries ltd
3,JP4059995A,27121457,1341188,4,1,0,JP4059995A,1995-02-28,1996-09-10,JPH08229330A,...,KR20100103462A\nJP2011502743A\nJP2013078764A\n...,KR20107010572A\nJP2010531202A\nJP2012279486A\n...,328440678\n332201837\n406999716\n378212937\n41...,40580359\n40580359\n40580359\n40580359\n405803...,31,SUMITOMO ELECTRIC IND LTD,SUMITOMO ELECTRIC IND LTD,PARTICULATE TRAP FOR DIESEL ENGINE,PURPOSE: To obtain a trap by which the leakage...,sumitomo electric ind ltd
4,WO9602405W,30070806,1341188,4,0,0,JP4059995A,1995-02-28,1998-03-05,WO9809059A1,...,NaN,NaN,NaN,NaN,31,"Sumitomo Electric Industries, Ltd.","Sumitomo Electric Industries, Ltd.",PARTICULATE TRAP FOR A DIESEL ENGINE,A particulate trap for a diesel engine designe...,sumitomo electric industries ltd


In [167]:
import pickle
patents_new.to_pickle('patent_firm_clean.pkl')

In [168]:
patent = pd.read_pickle('patent_firm_clean.pkl')

In [169]:
patent.head()

,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,publn,...,citing_publn,citing_appln,citing_appln_id,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr,assignee
0,WO9105114W,47158713,1239277,2,1,1,US55592190A,1990-07-19,1993-02-18,WO9303532A1,...,WO0199222A3,WO0119827W,45621927,24395918,30,LUZ ELECTRIC FUEL ISRAEL LTD.,LUZ ELECTRIC FUEL ISRAEL LTD.,ELECTRICAL ENERGY SYSTEM,An electrical energy system including an elect...,luz electric fuel israel ltd
1,US55592190A,51797454,1239277,2,1,1,US55592190A,1990-07-19,1992-06-09,US5121044A,...,US5208526A\nEP0557287A4\nUS5349535A\nUS5434020...,US63645090A\nEP91915464A\nUS96399792A\nUS15296...,52393850\n16916549\n54353461\n48212422\n457802...,27071024\n1239277\n25508001\n22545199\n2636904...,30,"Luz Electric Fuel Israel, Ltd.","Luz Electric Fuel Israel, Ltd.",Electrical energy system,An electrical energy system including an elect...,luz electric fuel israel ltd
2,EP96927918A,17205496,1341188,4,0,0,JP4059995A,1995-02-28,1998-08-12,EP0857862A1,...,EP1167708A2\nDE10029978A1\nDE10105233A1\nEP220...,EP01115227A\nDE10029978A\nDE10105233A\nEP08842...,15814252\n12936842\n12970055\n57178895,7646156\n7646156\n7672964\n40580359,31,"SUMITOMO ELECTRIC INDUSTRIES, LTD.","SUMITOMO ELECTRIC INDUSTRIES, LTD.",PARTICULATE TRAP FOR A DIESEL ENGINE,An object is to provide a particulate trap for...,sumitomo electric industries ltd
3,JP4059995A,27121457,1341188,4,1,0,JP4059995A,1995-02-28,1996-09-10,JPH08229330A,...,KR20100103462A\nJP2011502743A\nJP2013078764A\n...,KR20107010572A\nJP2010531202A\nJP2012279486A\n...,328440678\n332201837\n406999716\n378212937\n41...,40580359\n40580359\n40580359\n40580359\n405803...,31,SUMITOMO ELECTRIC IND LTD,SUMITOMO ELECTRIC IND LTD,PARTICULATE TRAP FOR DIESEL ENGINE,PURPOSE: To obtain a trap by which the leakage...,sumitomo electric ind ltd
4,WO9602405W,30070806,1341188,4,0,0,JP4059995A,1995-02-28,1998-03-05,WO9809059A1,...,NaN,NaN,NaN,NaN,31,"Sumitomo Electric Industries, Ltd.","Sumitomo Electric Industries, Ltd.",PARTICULATE TRAP FOR A DIESEL ENGINE,A particulate trap for a diesel engine designe...,sumitomo electric industries ltd


In [170]:
patent['assignee'] = patent['assignee'].str.replace(",","")

In [171]:
patent['assignee'] = patent['assignee'].str.replace(".","")

In [172]:
patent['assignee'].value_counts()[:100]

toyota motor corp                              7997
honda motor co ltd                             3899
nissan motor co ltd                            2670
robert bosch gmbh                              2659
hyundai motor co ltd                           2597
ford global technologies inc                   1671
mitsubishi motors corp                          899
renault sas                                     760
hitachi ltd                                     759
denso corp                                      640
peugeot citroen automobiles sa                  626
mitsubishi electric corp                        513
aisin aw co ltd                                 365
daimlerchrysler ag                              338
denso corporation                               337
continental automotive gmbh                     316
suzuki motor corp                               299
gm global technology operations llc             291
siemens aktiengesellschaft                      285
yamaha hatsu

In [173]:
p=patent

In [174]:
p.loc[p.assignee.str.contains('toyota jidosha',na=False)]

,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,publn,...,citing_publn,citing_appln,citing_appln_id,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr,assignee
5213,DE19701355A,13457689,11646668,4,1,0,JP674296A,1996-01-18,1997-07-24,DE19701355A1,...,DE19811574A1\nUS6401453B1\nDE10061685B4\nWO201...,DE19811574A\nUS66652700A\nDE10061685A\nWO20130...,13510156\n52611687\n12960865\n406380116\n33689...,7861206\n7861206\n18466322\n48536817\n44563021,16,"TOYOTA JIDOSHA K.K., TOYOTA, AICHI, JP","TOYOTA JIDOSHA K.K., TOYOTA, AICHI, JP",Catalytic converter for exhaust gases in vehic...,The catalytic converter (3) in the exhaust duc...,toyota jidosha kk toyota aichi jp
5352,JP1663492A,25858027,11921796,4,1,1,JP1663492A,1992-01-31,1993-08-20\n1999-05-10,JPH05209508A\nJP2888007B2,...,NaN,NaN,NaN,NaN,0,TOYOTA JIDOSHA KK,TOYOTA JIDOSHA KK,EXHAUST EMISSION CONTROL DEVICE FOR DIESEL ENGINE,PURPOSE:To perfectly carry out regeneration an...,toyota jidosha kk
5404,DE19901470A,13550451,12028753,12,1,1,JP2049598A,1998-01-16,1999-07-22\n2005-03-17,DE19901470A1\nDE19901470B4,...,WO0156824A1\nEP1211117A1\nWO03053730A1\nDE1020...,WO0101105W\nEP01000588A\nWO0203339W\nDE1020040...,15604711\n15801598\n9404250\n14923268\n1493936...,11457365\n26936991\n7708593\n34716556\n3463887...,116,"TOYOTA JIDOSHA K.K., TOYOTA","TOYOTA JIDOSHA K.K., TOYOTA",Drive control system for hybrid vehicle,A drive control system for a hybrid vehicle ha...,toyota jidosha kk toyota
5437,DE19505726A,13381314,12080783,8,1,1,JP2237194A,1994-02-21,1995-09-21\n2001-12-06,DE19505726A1\nDE19505726C2,...,DE19721526A1\nDE19826551C1\nDE19723790C2\nUS64...,DE19721526A\nDE19826551A\nDE19723790A\nUS71962...,13472609\n13521282\n13474257\n52996750\n130007...,7830237\n7870901\n7831615\n7870901\n7699147\n1...,41,"TOYOTA JIDOSHA K.K., TOYOTA","TOYOTA JIDOSHA K.K., TOYOTA",Control for generator in hybrid vehicle,A control unit for the generator output of a h...,toyota jidosha kk toyota
5457,JP2394992A,26243358,12124804,4,1,1,JP2394992A,1992-02-10,1993-08-31\n1998-08-27,JPH05223021A\nJP2789908B2,...,EP0825340A2\nUS5836293A\nUS5944003A\nUS5950607...,EP97113897A\nUS91024597A\nUS90833697A\nUS91024...,17229294\n54105124\n54096274\n54105111\n482513...,16643835\n16646061\n16605894\n16643835\n263564...,5,TOYOTA JIDOSHA KK,TOYOTA JIDOSHA KK,EVAPORATED FUEL DISPOSAL DEVICE FOR INTERNAL C...,PURPOSE:To provide an evaporated fuel disposal...,toyota jidosha kk
5625,JP3551791A,26853012,12443949,4,1,1,JP3551791A,1991-02-04,1992-09-10\n1999-07-26,JPH04255541A\nJP2924219B2,...,US6866610B2\nJP2007270995A\nUS7591130B2\nEP227...,US9280002A\nJP2006098486A\nUS3313605A\nEP10181...,47204376\n39223114\n46124988\n323763755\n32376...,27346431\n38674006\n27346431\n27346431\n273464...,2,TOYOTA JIDOSHA KK,TOYOTA JIDOSHA KK,CONTROL DEVICE FOR VEHICLE PROVIDED WITH LEAN-...,PURPOSE:To control a combustion condition of a...,toyota jidosha kk
5667,JP3866991A,27019511,12531682,4,1,1,JP3866991A,1991-03-05,1992-10-05\n1999-10-04,JPH04279746A\nJP2956237B2,...,JP2010071202A\nUS7729846B2,JP2008239991A\nUS56188309A,315732596\n275312565,42203217\n42006125,2,TOYOTA JIDOSHA KK,TOYOTA JIDOSHA KK,FUEL CHARACTER DETECTING DEVICE OF INTERNAL CO...,PURPOSE:To detect the character of an injectin...,toyota jidosha kk
5739,JP4382791A,27291712,12674590,4,1,1,JP4382791A,1991-03-08,1992-10-05\n1998-12-14,JPH04279748A\nJP2836270B2,...,US6431160B1,US67002000A,52636841,17713638,11,TOYOTA JIDOSHA KK,TOYOTA JIDOSHA KK,ABNORMALITY DIAGNOSIS DEVICE OF FUEL INJECTION...,PURPOSE:To prevent a deterioration of the exha...,toyota jidosha kk
5811,DE19708994A,13463330,12828575,4,0,0,JP4935096A,1996-03-06,1997-11-06,DE19708994A1,...,DE102012204698B4,DE102012204698A,375460484,46801812,4,"TOYOTA JIDOSHA K.K., TOYOTA, AICHI, JP","TOYOTA JIDOSHA K.K., TOYOTA, AICHI, JP",Control apparatus for motor vehicle,The control apparatus includes a throttle valv...,toyota jidosha kk toyota aichi jp
5833,

In [175]:
len(p.loc[p.assignee.str.contains('toyota',na=False)])

8359

In [176]:
len(p.loc[p.assignee.str.contains('toyota jidosha',na=False)])

153

### Coming back to cleaning more

In [177]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kk toyota aichi jp','toyota motor corp')

In [178]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kk','toyota motor corp')

In [179]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kabushiki kaisya','toyota motor corp')

In [180]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kakushiki kaisha','toyota motor corp')

In [181]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kabushikikaisha','toyota motor corp')

In [182]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kabushikikaisha','toyota motor corp')

In [183]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kubishiki kaisha','toyota motor corp')

In [184]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kanushiki kaisha','toyota motor corp')

In [185]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kabsuhiki kaisha','toyota motor corp')

In [186]:
p['assignee']=p['assignee'].str.replace("toyota industries corp","nissan motor co ltd")

In [187]:
p['assignee']=p['assignee'].str.replace('nissan motor','nissan motor co ltd')

In [188]:
p['assignee']=p['assignee'].str.replace("nissan motor co ltd co ltd","nissan motor co ltd")

In [189]:
p['assignee']=p['assignee'].str.replace("zhengzhou nissan automobile co ltd","nissan motor co ltd")

In [190]:
p['assignee']=p['assignee'].str.replace("nissan diesel motor co ltd","nissan motor co ltd")

In [191]:
p['assignee']=p['assignee'].str.replace("suzuki motor corporation","suzuki motor corp")

In [192]:
p['assignee']=p['assignee'].str.replace("honda giken kogyo kk","honda motor co ltd")

In [193]:
p['assignee']=p['assignee'].str.replace("honda motor coltd","honda motor co ltd")

In [194]:
p['assignee']=p['assignee'].str.replace("volkswagen aktiengesellschaft","volkswagen group")

In [195]:
p['assignee']=p['assignee'].str.replace("volkswagen ag","volkswagen group")

In [196]:
p['assignee']=p['assignee'].str.replace("volkswagen ag","volkswagen group")

In [197]:
p['assignee']=p['assignee'].str.replace("peugeot citroen automobiles sa","peugeot citroen automobiles")

In [198]:
p['assignee']=p['assignee'].str.replace("peugeot citroën automobiles sa","peugeot citroen automobiles")

In [199]:
p['assignee']=p['assignee'].str.replace("peugeot citroën automobiles sa","peugeot citroen automobiles")

In [200]:
p['assignee']=p['assignee'].str.replace("robert bosch gmbh","bosch corp")

In [201]:
p['assignee']=p['assignee'].str.replace("bosch gmbh robert","bosch corp")

In [202]:
p['assignee']=p['assignee'].str.replace("renault sas societe par actions simplifiee","renault sas")

In [203]:
p['assignee']=p['assignee'].str.replace("isuzu motors limited","isuzu motors ltd")

In [204]:
p['assignee']=p['assignee'].str.replace("aisin seiki co ltd","aisin aw co ltd")

In [205]:
p['assignee']=p['assignee'].str.replace("denso corporation","denso corp")

In [206]:
p['assignee']=p['assignee'].str.replace("nippondenso co ltd","denso corp")

In [207]:
p['assignee']=p['assignee'].str.replace("bayerische motoren werke aktiengesellschaft","bmw ag")

In [208]:
p['assignee']=p['assignee'].str.replace("bayerische motoren werke (bmw) ag","bmw ag")

In [209]:
p['assignee']=p['assignee'].str.replace("BMW AG","bmw ag")

In [210]:
p.loc[p.assignee.str.contains('bmw',na=False)]

,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,publn,...,citing_publn,citing_appln,citing_appln_id,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr,assignee
333,EP90124706A,16827458,6401764,4,1,1,DE4007438A,1990-03-09,1991-09-11\n1994-02-02,EP0445407A1\nEP0445407B1,...,WO0029246A3\nDE102006045403A1,WO9903621W\nDE102006045403A,11714153\n15051545,26050197\n39134138,9,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,"Drive unit, especially for driving vehicles.","A drive assemby, in particular for driving a v...",bmw ag
667,DE4223782A,11230847,6463602,4,1,1,DE4223782A,1992-07-18,1994-01-20\n2010-05-06,DE4223782A1\nDE4223782B4,...,FR2757900A1\nEP0924418A3,FR9616213A\nEP98123457A,20536344\n333937,9499299\n18394357,2,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,Air intake control system for car IC engine - ...,The control system has a variable ratio of the...,bmw ag
706,EP93112017A,16987950,6468829,4,1,1,DE4232148A,1992-09-25,1994-03-30\n1994-08-03\n1997-04-16,EP0589176A2\nEP0589176A3\nEP0589176B1,...,WO9614561A1\nDE19818697A1\nWO0006886A1\nEP0952...,WO9401104W\nDE19818697A\nWO9901086W\nEP9910753...,43575875\n13515499\n11711704\n17378903\n411653...,20395849\n7865888\n7875529\n7865888\n19731870\...,11,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,Method for testing the tightness of a tank ins...,In a method for testing the tightness of a tan...,bmw ag
807,EP93116401A,16991809,6483087,4,1,1,DE4308570A,1993-03-18,1994-09-21\n1996-07-24,EP0616120A1\nEP0616120B1,...,NaN,NaN,NaN,NaN,0,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,Internal combustion engine with lambda control...,The valve is located in a tube section in the ...,bmw ag
826,EP94101610A,17032605,6485134,4,1,1,DE4311697A,1993-04-08,1994-10-26\n1996-07-24,EP0621152A1\nEP0621152B1,...,FR2798531A1\nFR2802592A1\nEP1112884A1\nFR28034...,FR0011389A\nFR0016709A\nEP00403476A\nFR9916641...,19006243\n19011081\n15746806\n20619711\n190181...,7921314\n34219570\n9553957\n9553957\n8861202\n...,28,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,"Drive arrangement for motor vehicles, especial...",The invention relates to a drive arrangement f...,bmw ag
905,EP94111958A,17041729,6495553,4,1,1,DE4327906A,1993-08-19,1995-02-22\n1999-01-20,EP0639476A1\nEP0639476B1,...,NaN,NaN,NaN,NaN,12,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,Method of reducing combustion engine torque du...,In a device for reducing the combustion engine...,bmw ag
956,EP94115852A,17045189,6501834,4,1,1,DE4337720A,1993-11-05,1995-05-10\n1996-01-10\n1998-06-10,EP0652425A2\nEP0652425A3\nEP0652425B1,...,DE102006008062B3\nDE102006020434A1\nUS7962273B...,DE102006008062A\nDE102006020434A\nUS28005407A\...,15023551\n15032782\n55794261\n375087779\n14956...,37950182\n38564779\n37950182\n46756567\n360115...,8,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,Device for evaluating the compression of a mul...,In a device for evaluating the compression of ...,bmw ag
1086,EP95107227A,17094267,6521500,4,1,1,DE4422272A,1994-06-24,1995-12-27\n1998-10-14,EP0688942A1\nEP0688942B1,...,WO2014080278A3\nUS9581076B2,WO2013002790W\nUS201314397885A,413410648\n444210776,49918739\n49918739,3,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,Cooling device for a liquid-cooled internal co...,The IC engine (10) is cooled by a liquid circu...,bmw ag
1255,EP01101851A,216448,7630187,4,1,1,DE10005474A,2000-02-08,2001-08-16\n2004-08-04\n2007-11-21,EP1124051A2\nEP1124051A3\nEP1124051B1,...,US5607345A,US18119594A,48447034,22663278,2,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,Method and device for desulphating a nox acc

In [211]:
p['assignee'].value_counts()[:60]

toyota motor corp                                     8125
honda motor co ltd                                    3977
nissan motor co ltd                                   2885
bosch corp                                            2770
hyundai motor co ltd                                  2597
ford global technologies inc                          1671
denso corp                                            1097
mitsubishi motors corp                                 899
renault sas                                            859
hitachi ltd                                            759
peugeot citroen automobiles                            688
suzuki motor corp                                      528
mitsubishi electric corp                               513
aisin aw co ltd                                        458
volkswagen group                                       375
daimlerchrysler ag                                     338
continental automotive gmbh                            3

### Final work on Ford & yamaha

In [212]:
p['assignee']=p['assignee'].str.replace("ford motor company","ford global technologies inc")

In [213]:
p['assignee']=p['assignee'].str.replace("kaisha yamaha motor co ltd","yamaha hatsudoki kabushiki kaisha")

In [220]:
p['assignee']=p['assignee'].str.replace("yamaha hatsudoki kabushiki kaisha","yamaha motor co ltd")

In [214]:
p['assignee']=p['assignee'].str.replace("mitsubishi electric corp","mitsubishi motors corp")

In [215]:
len(p[p.assignee =='yamaha hatsudoki kabushiki kaisha'])

280

In [216]:
len(p[p.assignee =='kaisha yamaha motor co ltd'])

0

In [221]:
len(p[p.assignee =='yamaha motor co ltd'])

466

In [217]:
len(p[p.assignee =='mitsubishi electric corp'])

0

In [114]:
len(p[p.assignee =='mitsubishi motors corp'])

1412

In [ ]:
yamaha hatsudoki kabushiki kaisha   kaisha yamaha motor co ltd yamaha motor co ltd

In [222]:
p['assignee'].value_counts()[:60]

toyota motor corp                                     8125
honda motor co ltd                                    3977
nissan motor co ltd                                   2885
bosch corp                                            2770
hyundai motor co ltd                                  2597
ford global technologies inc                          1813
mitsubishi motors corp                                1412
denso corp                                            1097
renault sas                                            859
hitachi ltd                                            759
peugeot citroen automobiles                            688
suzuki motor corp                                      528
yamaha motor co ltd                                    466
aisin aw co ltd                                        458
volkswagen group                                       375
daimlerchrysler ag                                     338
continental automotive gmbh                            3

In [223]:
p.to_pickle('patent_firm_full_clean.pkl')

### complete work